In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/customer-personality-analysis/marketing_campaign.csv


In [2]:
#Importing the Libraries
import numpy as np
import pandas as pd
import datetime
from datetime import datetime
import matplotlib

In [3]:
#Loading the dataset
data = pd.read_csv("../input/customer-personality-analysis/marketing_campaign.csv", sep="\t")
print("Number of datapoints:", len(data))
data.head()

Number of datapoints: 2240


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [4]:
#Information on features 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

In [5]:
#Remove the NA values
data = data.dropna()
print("The total number of data-points after removing the rows with missing values are:", len(data))

The total number of data-points after removing the rows with missing values are: 2216


The total number of data-points after removing the rows with missing values are: 2216

In [6]:
#Find Min Year_Birth
df = pd.DataFrame(data)
min_year = df['Year_Birth'].min()
print(f"The minimum year of birth is {min_year}.")

The minimum year of birth is 1893.


Since our dataset contains 1893, I decided to find all years <= 1900.

In [7]:
#Find all rows where Year_Birth <= 1900
df = pd.DataFrame(data)
years_before_1900 = df[df['Year_Birth'] <= 1900]
print(years_before_1900)

        ID  Year_Birth Education Marital_Status   Income  Kidhome  Teenhome  \
192   7829        1900  2n Cycle       Divorced  36640.0        1         0   
239  11004        1893  2n Cycle         Single  60182.0        0         1   
339   1150        1899       PhD       Together  83532.0        0         0   

    Dt_Customer  Recency  MntWines  ...  NumWebVisitsMonth  AcceptedCmp3  \
192  26-09-2013       99        15  ...                  5             0   
239  17-05-2014       23         8  ...                  4             0   
339  26-09-2013       36       755  ...                  1             0   

     AcceptedCmp4  AcceptedCmp5  AcceptedCmp1  AcceptedCmp2  Complain  \
192             0             0             0             0         1   
239             0             0             0             0         0   
339             0             1             0             0         0   

     Z_CostContact  Z_Revenue  Response  
192              3         11         0  
2

There are 3 rows where Year_Birth <=1900.
Going to remove these rows.

In [8]:
#Filtering the values where Year_Birth >1900
data = pd.DataFrame(data)
data = data[data['Year_Birth'] > 1900]
print("The total number of data-points after removing the rows with missing values are:", len(data))

The total number of data-points after removing the rows with missing values are: 2213


The total number of data-points after removing the rows with missing values are: 2213

As the data set had values where Year_Birth <= 1900.
Decided to check the max Year_Birth

In [9]:
#Find Max Year
df = pd.DataFrame(data)
max_year = df['Year_Birth'].max()
print(f"The maximum year of birth is {max_year}.")

The maximum year of birth is 1996.


As the maximum Year_Birth is 1996, all other Year_Birth values are good to use in the next analysis.

And I would like to know what is the number of clients of each age?
First, I want to calculate number of years for customers customers. 

In [10]:
#Number of customers for each age

#Calculation of age
current_year = datetime.now().year
data['Age'] = current_year - data['Year_Birth']

# Group by 'Age' and count the number of 'ID' for each age group
grouped = data.groupby('Age')['ID'].count().reset_index(name='Number_customers')

# Sort by 'Age' in descending order
grouped = grouped.sort_values(by='Age')

print("Total customers by ages:\n", grouped, "\n")

Total customers by ages:
     Age  Number_customers
0    29                 2
1    30                 5
2    31                 3
3    32                 5
4    33                13
5    34                15
6    35                18
7    36                29
8    37                29
9    38                27
10   39                41
11   40                32
12   41                38
13   42                41
14   43                44
15   44                38
16   45                39
17   46                53
18   47                76
19   48                52
20   49                89
21   50                83
22   51                69
23   52                72
24   53                78
25   54                86
26   55                75
27   56                70
28   57                51
29   58                44
30   59                50
31   60                74
32   61                41
33   62                44
34   63                44
35   64                35
36   65     

My next step is find a median age. 
The median is the middle value when a data set is ordered from least to greatest

In [11]:
#Calculation of median age

df = pd.DataFrame(data)
median_age = df['Age'].median()
print("The median age is:\n", median_age, "\n")

The median age is:
 55.0 



The median age is: 55 years. 

In [12]:
#Calculation number of customers for the mode age

age_with_max_customers = grouped.loc[grouped['Number_customers'].idxmax()]
print("The age with the highest number of customers is:\n", age_with_max_customers, "\n")

The age with the highest number of customers is:
 Age                 49
Number_customers    89
Name: 20, dtype: int64 



There are 89 customers belong to mode age, which is 49 years.

Now we will be exploring the unique values in the categorical features to get a clear idea of the data.  

In [13]:
print("Total categories in the feature Marital_Status:\n", data["Marital_Status"].value_counts(), "\n")
print("Total categories in the feature Education:\n", data["Education"].value_counts())

Total categories in the feature Marital_Status:
 Married     857
Together    572
Single      470
Divorced    231
Widow        76
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64 

Total categories in the feature Education:
 Graduation    1116
PhD            480
Master         365
2n Cycle       198
Basic           54
Name: Education, dtype: int64


**Performed the following steps to engineer some new features:**

* Extract the **"Age"** of a customer by the **"Year_Birth"** indicating the birth year.
* Create another feature **"Spent"** indicating the total amount spent by the customer in various categories over the span of two years.
* Create another feature **"Living_With"** out of **"Marital_Status"** to extract the living situation of couples.
* Create a feature **"Children"** to indicate total children in a household that is, kids and teenagers.
* To get further clarity of household, Creating feature indicating **"Family_Size"**
* Create a feature **"Is_Parent"** to indicate parenthood status
* Create three categories in the **"Education"** by simplifying its value counts.
* Dropping some of the redundant features

In [14]:
#Feature Engineering

#Age of customer today 
data["Age"] = (datetime.now().year)-data["Year_Birth"]

#Total spendings on various items
data["Spent"] = data["MntWines"]+ data["MntFruits"]+ data["MntMeatProducts"]+ data["MntFishProducts"]+ data["MntSweetProducts"]+ data["MntGoldProds"]

#Deriving living situation by marital status"Alone"
data["Living_With"]=data["Marital_Status"].replace({"Married":"Partner", "Together":"Partner", "Absurd":"Alone", "Widow":"Alone", "YOLO":"Alone", "Divorced":"Alone", "Single":"Alone",})

#Feature indicating total children living in the household
data["Children"]=data["Kidhome"]+data["Teenhome"]

#Feature for total members in the householde
data["Family_Size"] = data["Living_With"].replace({"Alone": 1, "Partner":2})+ data["Children"]

#Feature pertaining parenthood
data["Is_Parent"] = np.where(data.Children> 0, 1, 0)

#Segmenting education levels in three groups
data["Education"]=data["Education"].replace({"Basic":"Undergraduate","2n Cycle":"Undergraduate", "Graduation":"Graduate", "Master":"Postgraduate", "PhD":"Postgraduate"})

#For clarity
data=data.rename(columns={"MntWines": "Wines","MntFruits":"Fruits","MntMeatProducts":"Meat","MntFishProducts":"Fish","MntSweetProducts":"Sweets","MntGoldProds":"Gold"})

#Dropping some of the redundant features
to_drop = ["Marital_Status", "Dt_Customer", "Z_CostContact", "Z_Revenue", "Year_Birth", "ID"]
data = data.drop(to_drop, axis=1)

Now that we have some new features let's have a look at the data's stats. 

data.describe()